In [1]:
# Form our test and train data
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

# Form our test and train data
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
import random 

from keras.callbacks import EarlyStopping, ModelCheckpoint
from collections import defaultdict

### Data generation

In [40]:
def check_valie(data_seq):
    if len(data_seq) > 1:
        if data_seq[-2] == data_seq[-1] == 1:
            print("Check success")
            return False
        else:
            return True
    else:
        return True  
def create_train_data(length):
    train_size = 1000
    x_train = np.zeros((train_size, length), dtype=int)
    y_train =  np.zeros(train_size, dtype=int)
    # Generating class 1 data
    for i in range(train_size//2):
        data_class1 = []
        if random.randrange == 1:
            data_class1.append(1)
        while len(data_class1) < length:
            data_class1.extend([0]*random.randrange(1, 11, 1))
            data_class1.append(1)
        x_train[2*i] = np.array(data_class1[:length])
        y_train[2*i] = 1
        print("Finish creating data 1...")
      
        # Generatring class 2 data:
        flag = True
        while flag:
            data_class2 = []
            if random.randrange == 1:
                data_class2.extend([1]*random.randrange(1, 11))
                flag = check_valie(data_class2) if flag else False
                
            while len(data_class2) < length:
                data_class2.extend([0]*random.randrange(11))
                data_class2.extend([1]*random.randrange(1, 11))
                flag = check_valie(data_class2)  if flag else False
        x_train[2*i+1] = np.array(data_class2[:length])
        print("Finish creating data 2...")
    combined = list(zip(x_train, y_train))
    random.shuffle(combined)
    print("Finish shuffling...")
    # Unzip the pairs back into two lists
    return zip(*combined)

In [41]:
L = 60   # Define the length of the sequence]
train_data, train_label = create_train_data(L)

Finish creating data 1...
Check success
Finish creating data 2...
Finish creating data 1...
Check success
Finish creating data 2...
Finish creating data 1...
Check success
Finish creating data 2...
Finish creating data 1...
Check success
Finish creating data 2...
Finish creating data 1...
Check success
Finish creating data 2...
Finish creating data 1...
Check success
Finish creating data 2...
Finish creating data 1...
Check success
Finish creating data 2...
Finish creating data 1...
Check success
Finish creating data 2...
Finish creating data 1...
Check success
Finish creating data 2...
Finish creating data 1...
Check success
Finish creating data 2...
Finish creating data 1...
Check success
Finish creating data 2...
Finish creating data 1...
Check success
Finish creating data 2...
Finish creating data 1...
Check success
Finish creating data 2...
Finish creating data 1...
Check success
Finish creating data 2...
Finish creating data 1...
Check success
Finish creating data 2...
Finish cre

In [52]:
with open('data/train_data.txt', 'w') as file:
    for data_list in train_data:
        line = ','.join(str(num) for num in data_list)
        file.write(line + '\n')

with open('data/train_label.txt', 'w') as file:
    file.write(','.join(map(str, train_label)))

### Data Loading

In [9]:
with open('data/train_data.txt', 'r') as file:      # Read train dataset
    train_data = np.array([list(map(int, line.split(','))) for line in file], dtype=int)
  
with open('data/train_label.txt', 'r') as file:     # Read train label
    train_label = np.array([int(i) for i in file.read().split(',')], dtype=int)  # Split the string back into a list

In [6]:
# split = int(0.8*len(train_data))
# x_train, x_val = train_data[:split], train_data[split:]
# y_train, y_val = train_label[:split], train_label[split:]

In [7]:
with open('data/hmw5test.txt', 'r') as file:      # Read train dataset
    test_data_label = np.array([list(map(int, line.split(','))) for line in file], dtype=int)

test_data = test_data_label[:, :-1]
test_label = test_data_label[:, -1]

In [27]:
def string_to_one_hot(strings, max_length=60):
    one_hot_encoded = []
    for string in strings:
        one_hot_encoded.append(to_categorical(string, num_classes=2))
    return np.array(one_hot_encoded, dtype=int)

In [30]:
testing = string_to_one_hot(np.array(train_data, dtype=int))

(array([[1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0]]),
 [0,
  0,
  1,
  0,
  0,
  0,
  0,
  0

### RNN Model training

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout

def build_RNN(input_shape, units=64):
    callbacks = [EarlyStopping(monitor='val_accuracy', patience=20),
                 ModelCheckpoint(filepath='model.h5', monitor='val_accuracy', save_best_only=True)]
    
    model = Sequential()
    model.add(SimpleRNN(units, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [16]:
train_data.shape

(1000, 60)

In [15]:
model = build_RNN(train_data.shape)
history = model.fit(train_data, train_label, epochs=200, batch_size=64, validation_split=0.2)

Epoch 1/200


ValueError: in user code:

    File "C:\ProgramData\Anaconda3\envs\contest\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\ProgramData\Anaconda3\envs\contest\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\ProgramData\Anaconda3\envs\contest\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\ProgramData\Anaconda3\envs\contest\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "C:\ProgramData\Anaconda3\envs\contest\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\ProgramData\Anaconda3\envs\contest\lib\site-packages\keras\engine\input_spec.py", line 264, in a

In [ ]:
performace = model.evaluate(test_data, test_label)
print(f"Test Loss: {performance[0]}, Test Accuracy: {performance[1]}")